<a href="https://colab.research.google.com/github/vcoelh/Algoritmos-e-Estrutura-de-Dados/blob/main/robo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import pandas as pd
from datetime import date, datetime
from time import sleep
from IPython.display import display
import os

class WebScraper:
    def __init__(self):
        self.navegador = self.iniciar_driver()

    def pegar_title(self, EAN_t='.product-title'):
        nome_elementos = self.navegador.find_element(By.CSS_SELECTOR, EAN_t).text
        return nome_elementos

    def pegar_preco(self, EAN_p='.product-price'):
        preco_elemetos = self.navegador.find_element(By.CSS_SELECTOR, EAN_p)
        return preco_elemetos.text.split('R$')[-1]

    def pegar_url(self, EAN_u='.product-title > a'):
        default_category = 'https://www.marchio.com/shop/category/ferramentaseletricas/'
        lista_url = []
        i = 0
        while True:
            i += 1
            self.navegador.get(f'{default_category}?page={i}')
            sleep(1)
            boxs = self.navegador.find_elements(By.CSS_SELECTOR, EAN_u)

            for url in boxs:
                url = url.get_attribute('href')
                lista_url.append(url)

            if i == 2:
                break

        print(lista_url)
        return lista_url

    def print_screen(self, codigos, nome, folder_screenshot="screenshot"):
        current_dir = os.path.join(os.getcwd(), folder_screenshot)
        os.makedirs(current_dir, exist_ok=True)

        nome = nome.split(" ")[0]
        try:
            nome = nome.replace("/", "-")
        except:
            pass
        codigo_produto_nome_produto_coleta_data_coleta = f'\\{codigos}_{nome}_{datetime.today().strftime("%d_%m_%Y")}_coleta.png'

        file_image = (os.path.abspath(folder_screenshot) + codigo_produto_nome_produto_coleta_data_coleta)
        print(file_image)
        self.navegador.save_screenshot(file_image)

    def pegar_cod(self, EAN_i='.product-details p '):
        imagem_elementos = self.navegador.find_element(By.CSS_SELECTOR, EAN_i)
        return imagem_elementos.text.split('Cod.:')[-1]

    def iniciar_driver(self):
        servico = Service(ChromeDriverManager().install())
        navegador = webdriver.Chrome(service=servico)
        return navegador

    def iterations(self):
        dados = {'nome_produto_coleta': [], 'codigo_produto': [], 'preco_coleta': [], 'url_coleta': [], 'data_coleta': []}
        link = self.pegar_url()
        dados['url_coleta'].extend(link)

        for url in link:
            self.navegador.get(url)
            nome = self.pegar_title()
            dados['nome_produto_coleta'].append(nome)

            preco = self.pegar_preco()
            dados['preco_coleta'].append(preco)

            codigos = self.pegar_cod()
            dados['codigo_produto'].append(codigos)

            self.print_screen(codigos=codigos, nome=nome, folder_screenshot='screenshot')

        data_coleta = [date.today()] * len(dados['preco_coleta'])
        dados['data_coleta'].extend(data_coleta)

        self.navegador.quit()

        df = pd.DataFrame(dados)
        return df

if __name__ == '__main__':
    scraper = WebScraper()
    dado = scraper.iterations()
    display(dado)


ModuleNotFoundError: ignored